**Commands to start Development Environment**
1. Install Docker-Desktop, Login
2. docker run (Jupyter notebook - should update command once loggeing out)
    1. docker run --name devenv -it --entrypoint=/bin/bash -p 8880:8888 -v C:\users\srmenon\Documents:/notebooks jupyter/tensorflow-note
    
3. cd /notebooks/
4. nohup jupyter notebook --port 8880 --allow-root &

In [ ]:
#!pip install twitter
#!pip install feedparser

In [ ]:
# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))
t.search.tweets(q="$MSFT")

In [ ]:
# Download latest info about ticker from Y! Finance Feed 
# https://query1.finance.yahoo.com/v7/finance/chart/MSFT?&interval=5m
import json
from urllib.request import urlopen
json.load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/MSFT?&interval=5m"))

In [ ]:
# Use feedparser to download and parse reuters feed for business news
# https://www.pythonforbeginners.com/feedparser/using-feedparser-in-python
# http://feeds.reuters.com/reuters/businessNews
import feedparser
d = feedparser.parse("http://feeds.reuters.com/reuters/businessNews")
print([l.title for l in d["entries"]])

**Commands to start Kafka Cluster**

https://success.docker.com/article/getting-started-with-kafka

Use linux container mode (else bridging will fail for overlay network)

1. docker swarm init
2. docker network create -d overlay --attachable kafka-net
3. docker service create --network kafka-net --name=zookeeper  --publish 2181:2181 qnib/plain-zookeeper:latest
4. docker service create --network kafka-net --name=zkui --publish 9090:9090 qnib/plain-zkui:latest
5. Visit http://localhost:9090/  # Login using admin/manager
6. docker service create --network kafka-net --name broker --publish 9092:9092 --hostname="{{.Service.Name}}.{{.Task.Slot}}.{{.Task.ID}}" -e KAFKA_BROKER_ID={{.Task.Slot}} -e ZK_SERVERS=tasks.zookeeper qnib/plain-kafka:latest
7. docker ps -q --filter "label=com.docker.swarm.service.name=broker"
8. docker exec -t -e JMX_PORT="" f6dc03b145c0 /opt/kafka/bin/kafka-topics.sh --zookeeper tasks.zookeeper:2181 --partitions=1 --replication-factor=1 --create --topic test
9. docker service create --network kafka-net --name manager -e ZOOKEEPER_HOSTS=tasks.zookeeper --publish=9000:9000 qnib/plain-kafka-manager:latest

http://localhost:9000/clusters/kafka

OR EQUIVALENTLY

https://docs.docker.com/compose/gettingstarted/

1. docker-compose up -d KafkaSetup.yml 

In [ ]:
!pip install kafka-python

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers=['broker.1.pj1avromrejlg3vk5ab8iq9er:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

for e in range(1000):
    print("Processing "+str(e)+"\n")
    data = {'number' : e}
    producer.send('test', value=data)
    sleep(5)

In [ ]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer(
    'test',
     bootstrap_servers=['broker.1.pj1avromrejlg3vk5ab8iq9er:9092'],
     api_version= (0,11),
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

for message in consumer:
    message = message.value
    print('{} added'.format(message))